In [1]:
import duckdb

In [2]:
file_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/itineraries_snappy.parquet"

In [3]:
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')")

In [7]:
row_count = con.execute("SELECT COUNT(*) FROM flights").fetchall()
print(f"Total rows in the dataset: {row_count[0][0]}")

Total rows in the dataset: 82138753


In [8]:
columns= con.execute("DESCRIBE flights").fetchall()

for column in columns:
    print(f"Column Name: {column[0]}, Data Type: {column[1]}")

Column Name: legId, Data Type: VARCHAR
Column Name: searchDate, Data Type: VARCHAR
Column Name: flightDate, Data Type: VARCHAR
Column Name: startingAirport, Data Type: VARCHAR
Column Name: destinationAirport, Data Type: VARCHAR
Column Name: fareBasisCode, Data Type: VARCHAR
Column Name: travelDuration, Data Type: VARCHAR
Column Name: elapsedDays, Data Type: BIGINT
Column Name: isBasicEconomy, Data Type: BOOLEAN
Column Name: isRefundable, Data Type: BOOLEAN
Column Name: isNonStop, Data Type: BOOLEAN
Column Name: baseFare, Data Type: DOUBLE
Column Name: totalFare, Data Type: DOUBLE
Column Name: seatsRemaining, Data Type: BIGINT
Column Name: totalTravelDistance, Data Type: DOUBLE
Column Name: segmentsDepartureTimeEpochSeconds, Data Type: VARCHAR
Column Name: segmentsDepartureTimeRaw, Data Type: VARCHAR
Column Name: segmentsArrivalTimeEpochSeconds, Data Type: VARCHAR
Column Name: segmentsArrivalTimeRaw, Data Type: VARCHAR
Column Name: segmentsArrivalAirportCode, Data Type: VARCHAR
Column N

In [4]:
con.execute("""
    CREATE TABLE summer_flights AS 
    SELECT 
        legId,
        searchDate,
        flightDate,
        startingAirport,
        destinationAirport,
        fareBasisCode,
        travelDuration,
        isBasicEconomy,
        isRefundable,
        isNonStop,
        baseFare,
        totalFare,
        seatsRemaining,
        totalTravelDistance,
        segmentsDepartureTimeEpochSeconds,
        segmentsArrivalTimeEpochSeconds,
        segmentsArrivalAirportCode,
        segmentsDepartureAirportCode,
        segmentsAirlineName,
        segmentsDurationInSeconds,
        segmentsDistance,
        segmentsCabinCode
    FROM flights
    WHERE EXTRACT(MONTH FROM CAST(flightDate as DATE)) IN (6, 7, 8)
"""
)

In [11]:
row_count = con.execute("SELECT COUNT(*) FROM summer_flights").fetchall()
print(f"Total rows in summer flights dataset: {row_count[0][0]}")


Total rows in summer flights dataset: 42946320


In [5]:
output_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/summer_flights_snappy.parquet"
con.execute(f"COPY (SELECT * FROM summer_flights) TO '{output_path}' (FORMAT PARQUET, COMPRESSION SNAPPY)")

# Close the connection
con.close()

### California airport selection (LAX, SFO, OAK)
~17 Million entries 


In [7]:
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')") 

# Modify the query to filter for California airports (LAX, SFO, OAK)
con.execute("""
    CREATE TABLE summer_california_flights AS 
    SELECT 
        legId,
        searchDate,
        flightDate,
        startingAirport,
        destinationAirport,
        fareBasisCode,
        travelDuration,
        isBasicEconomy,
        isRefundable,
        isNonStop,
        baseFare,
        totalFare,
        seatsRemaining,
        totalTravelDistance,
        segmentsDepartureTimeEpochSeconds,
        segmentsArrivalTimeEpochSeconds,
        segmentsArrivalAirportCode,
        segmentsDepartureAirportCode,
        segmentsAirlineName,
        segmentsDurationInSeconds,
        segmentsDistance,
        segmentsCabinCode
    FROM flights
    WHERE EXTRACT(MONTH FROM CAST(flightDate as DATE)) IN (6, 7, 8)
    AND (startingAirport IN ('LAX', 'SFO', 'OAK') OR destinationAirport IN ('LAX', 'SFO', 'OAK'))
""")

# Count the rows in the new California-focused summer dataset
row_count = con.execute("SELECT COUNT(*) FROM summer_california_flights").fetchall()
print(f"Total rows in California summer flights dataset: {row_count[0][0]}")

# Export the filtered dataset
output_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/summer_california_flights_snappy.parquet"
con.execute(f"COPY (SELECT * FROM summer_california_flights) TO '{output_path}' (FORMAT PARQUET, COMPRESSION SNAPPY)")

# Close the connection
con.close()


Total rows in California summer flights dataset: 17358555


### Single Airport Selection (LAX)
~8 Million Entries 


In [9]:
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')") 

con.execute("""
    CREATE TABLE summer_selected_airports AS 
    SELECT 
        legId,
        searchDate,
        flightDate,
        startingAirport,
        destinationAirport,
        fareBasisCode,
        travelDuration,
        isBasicEconomy,
        isRefundable,
        isNonStop,
        baseFare,
        totalFare,
        seatsRemaining,
        totalTravelDistance,
        segmentsDepartureTimeEpochSeconds,
        segmentsArrivalTimeEpochSeconds,
        segmentsArrivalAirportCode,
        segmentsDepartureAirportCode,
        segmentsAirlineName,
        segmentsDurationInSeconds,
        segmentsDistance,
        segmentsCabinCode
    FROM flights
    WHERE EXTRACT(MONTH FROM CAST(flightDate as DATE)) IN (6, 7, 8)
    AND (startingAirport IN ('LAX') OR destinationAirport IN ('LAX'))
""")

# Get the count of rows in the filtered dataset
row_count = con.execute("SELECT COUNT(*) FROM summer_selected_airports").fetchall()
print(f"Total rows in LAX summer flights dataset: {row_count[0][0]}")

# Export the filtered dataset to a Parquet file
output_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/summer_lax_snappy.parquet"
con.execute(f"COPY (SELECT * FROM summer_selected_airports) TO '{output_path}' (FORMAT PARQUET, COMPRESSION SNAPPY)")

# Close the DuckDB connection
con.close()

Total rows in LAX summer flights dataset: 8616415


In [11]:


# File path to the original dataset
file_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/itineraries_snappy.parquet"

# Connect to DuckDB and load the dataset
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')")

# Count flights for summer months (June, July, August) and for all airports (either as starting or destination airport)
airport_counts = con.execute("""
    SELECT airport, COUNT(*) AS flight_count 
    FROM (
        SELECT startingAirport AS airport, flightDate FROM flights 
        WHERE EXTRACT(MONTH FROM TRY_CAST(flightDate AS DATE)) IN (6, 7, 8)
        UNION ALL
        SELECT destinationAirport AS airport, flightDate FROM flights 
        WHERE EXTRACT(MONTH FROM TRY_CAST(flightDate AS DATE)) IN (6, 7, 8)
    )
    GROUP BY airport
    ORDER BY flight_count DESC
""").fetchall()

# Print results
for airport, count in airport_counts:
    print(f"Airport: {airport}, Total Flights: {count}")

# Close the DuckDB connection
con.close()



Airport: LAX, Total Flights: 8616415
Airport: LGA, Total Flights: 6420753
Airport: BOS, Total Flights: 6137513
Airport: SFO, Total Flights: 6019303
Airport: DFW, Total Flights: 5952501
Airport: ORD, Total Flights: 5758980
Airport: CLT, Total Flights: 5543546
Airport: ATL, Total Flights: 5407034
Airport: MIA, Total Flights: 5240062
Airport: DEN, Total Flights: 5004160
Airport: PHL, Total Flights: 4929767
Airport: DTW, Total Flights: 4714075
Airport: JFK, Total Flights: 4542752
Airport: EWR, Total Flights: 4282662
Airport: IAD, Total Flights: 3734665
Airport: OAK, Total Flights: 3588452


In [12]:


# File path to the original dataset
file_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/itineraries_snappy.parquet"

# Connect to DuckDB and load the dataset
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')")

# Count the number of times each airport is the departing airport (startingAirport)
departure_counts = con.execute("""
    SELECT startingAirport AS airport, COUNT(*) AS departure_count 
    FROM flights
    WHERE EXTRACT(MONTH FROM TRY_CAST(flightDate AS DATE)) IN (6, 7, 8)  -- Summer months
    GROUP BY startingAirport
    ORDER BY departure_count DESC
""").fetchall()

# Count the number of times each airport is the arriving airport (destinationAirport)
arrival_counts = con.execute("""
    SELECT destinationAirport AS airport, COUNT(*) AS arrival_count 
    FROM flights
    WHERE EXTRACT(MONTH FROM TRY_CAST(flightDate AS DATE)) IN (6, 7, 8)  -- Summer months
    GROUP BY destinationAirport
    ORDER BY arrival_count DESC
""").fetchall()

# Print the results for departure counts
print("Departure Counts:")
for airport, count in departure_counts:
    print(f"Airport: {airport}, Departures: {count}")

# Print the results for arrival counts
print("\nArrival Counts:")
for airport, count in arrival_counts:
    print(f"Airport: {airport}, Arrivals: {count}")

# Close the DuckDB connection
con.close()


Departure Counts:
Airport: LAX, Departures: 4329731
Airport: LGA, Departures: 3168648
Airport: BOS, Departures: 3083056
Airport: SFO, Departures: 3050571
Airport: DFW, Departures: 2918327
Airport: ORD, Departures: 2829279
Airport: CLT, Departures: 2811265
Airport: ATL, Departures: 2729638
Airport: MIA, Departures: 2560279
Airport: DEN, Departures: 2522618
Airport: PHL, Departures: 2494111
Airport: DTW, Departures: 2398881
Airport: JFK, Departures: 2240627
Airport: EWR, Departures: 2106120
Airport: IAD, Departures: 1917607
Airport: OAK, Departures: 1785562

Arrival Counts:
Airport: LAX, Arrivals: 4286684
Airport: LGA, Arrivals: 3252105
Airport: BOS, Arrivals: 3054457
Airport: DFW, Arrivals: 3034174
Airport: SFO, Arrivals: 2968732
Airport: ORD, Arrivals: 2929701
Airport: CLT, Arrivals: 2732281
Airport: MIA, Arrivals: 2679783
Airport: ATL, Arrivals: 2677396
Airport: DEN, Arrivals: 2481542
Airport: PHL, Arrivals: 2435656
Airport: DTW, Arrivals: 2315194
Airport: JFK, Arrivals: 2302125
Airpo